## Data Collection:

In [1]:
import spacy
import pandas as pd
import nltk
import re
from PyPDF2 import PdfReader
import json
import re

In [ ]:
reader = PdfReader("/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/ouput/a.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
print(text)

## Modul Config

In [2]:
#create global varible to load extracted data
extracted_text = {}


# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# using matcher 
from spacy.matcher import Matcher

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)


## Data Extractions

In [ ]:
# extract phone Name

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
    
    
name = extract_name(text)
print(name)
extracted_text["fullName"] = name

In [36]:
def extract_portfolioLinkUrl(resume_text):
    nlp_text = nlp(resume_text)
    matcher = Matcher(nlp.vocab)
    pattern = [
        {'LOWER': 'github'},
        {'IS_PUNCT': True, 'OP': '?'},
        {'LOWER': 'com'},
        {'IS_PUNCT': True, 'OP': '?'},
        {'LOWER': {'IN': ['/', 'user', 'org']}},
        {'IS_PUNCT': True, 'OP': '?'},
        {'LOWER': {'REGEX': '[a-zA-Z0-9_-]+'}},
        {'IS_PUNCT': True, 'OP': '?'},
        {'LOWER': {'REGEX': '(repos|tree|blob|issues|pull)'}, 'OP': '?'},
        {'IS_PUNCT': True, 'OP': '?'},
        {'LOWER': {'REGEX': '[a-zA-Z0-9/._-]+'}}
    ]
    matcher.add('PORTFOLIO', None, pattern)
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
    return None

txt = "https://github.com/kbrajwani"

portfolioLinkUrl = extract_portfolioLinkUrl(txt)
print(portfolioLinkUrl)
extracted_text["portfolioLinkUrl"] = portfolioLinkUrl


None


In [ ]:
# extract email 

import re
def get_email_addresses(string):
    r = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    return r.findall(string)

email = get_email_addresses(text)
print(email)

extracted_text["email"] = email

In [ ]:
import re

def get_urlLinkedin(string):
    pattern = re.compile(r'\b(?:https?://)?(?:www\.)?linkedin\.com/\S+\b')
    return pattern.findall(string)

urlLinkedin = get_urlLinkedin(text)
print(urlLinkedin) # prints ['linkedin.com/in/fikogunawan/']

extracted_text = {}
extracted_text["linkedinUrl"] = urlLinkedin


In [ ]:
# extract phone number 

def get_phone_numbers(string):
    r = re.compile(r'(\b\d{8,}\b)')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

phone_number= get_phone_numbers(text)
print(phone_number)

extracted_text["phoneNumber"] = phone_number

In [ ]:
# extract skils

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/support/skills.csv") 
    
    # extract values
    skills = list(data.columns.values)
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for chunk in nlp_text.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in skills:
            skillset.append(chunk_text)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


skills = extract_skills(text)
print(skills)

extracted_text["skilss"] = skills


In [ ]:
def extract_web(text):

    pattern = re.compile(
        r'"\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b"'
    )

    # Find matches for the pattern in the parsed text
    matches = re.findall(pattern, text)

    # Return the first match found, or None if no matches were found
    if matches:
        return matches[0]
    else:
        return None


# Example usage

web_url = extract_web(text)
print(web_url)
extracted_text["website"] = web_url

In [ ]:
import pycld2 as cld2
import langcodes

def extract_languages(text):
    isReliable, textBytesFound, details = cld2.detect(text)
    languages = [lang[1] for lang in details if lang[3] > 0]  # Filter out languages with zero confidence
    countries = [langcodes.Language(lang).language_name() for lang in languages]
    return countries

extracted_text["languages"] = extract_languages(text)
print(extracted_text["languages"])


In [ ]:
import pandas as pd
import re
import json
import spacy

nlp = spacy.load("en_core_web_sm")

file = "/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/support/university.csv"

def extract_university(text, file):
    df = pd.read_csv(file, header=None)
    universities = [i.lower() for i in df[1]]
    college_name = []
    listex = universities
    listsearch = [text.lower()]

    for i in range(len(listex)):
        for ii in range(len(listsearch)):
            if re.findall(listex[i], re.sub(" +", " ", listsearch[ii])):
                college_name.append(listex[i])

    doc = nlp(text)

    # Extract location
    location = ""
    for ent in doc.ents:
        if ent.label_ == "GPE":
            location = ent.text
            break

    # Extract GPA
    gpa = ""
    for match in re.findall(r"GPA: (\d\.\d{2})", text):
        gpa = match
        break

    # Extract faculty and other information
    faculty = ""
    faculty_match = re.search(r"Bachelor of [a-zA-Z ]+ \d{4}-\d{4}", text)
    if faculty_match:
        faculty = faculty_match.group(0)
        other_info = text[faculty_match.end() :].strip()

    return college_name, location, gpa, faculty

# Extract values from extract_university()
college_name, location, gpa, faculty = extract_university(text, file)

if college_name:
    # Append the values to extracted_text
    extracted_text["education"] = {
        "organization": college_name,
        "location": location,
        "gpa": gpa,
        "faculty": faculty
    }

# Print the updated extracted_text dictionary
print(json.dumps(extracted_text, indent=4))


In [ ]:
def extract_work_experiences(resume_text):
    doc = nlp(resume_text)
    matcher = Matcher(nlp.vocab)

    # Define the pattern for matching work experiences
    pattern = [{'ENT_TYPE': 'ORG', 'OP': '?'},
               {'ENT_TYPE': 'ADJ', 'OP': '?'}, 
               {'ENT_TYPE': 'JOB_TITLE', 'OP': '?'}, 
               {'LOWER': 'at'},
               {'ENT_TYPE': 'ORG'},
               {'ENT_TYPE': 'ADJ', 'OP': '?'}, 
               {'ENT_TYPE': 'DATE', 'OP': '?'}]

    # Add the pattern to the matcher
    matcher.add('EXPERIENCE', None, pattern)

    # Find all matches in the document
    matches = matcher(doc)

    # Extract the relevant information from the matches
    work_experiences = []
    for match_id, start, end in matches:
        work_experience = {}
        span = doc[start:end]
        company_name = [ent.text for ent in span.ents if ent.label_ == 'ORG']
        job_title = [ent.text for ent in span.ents if ent.label_ == 'JOB_TITLE']
        employment_dates = [ent.text for ent in span.ents if ent.label_ == 'DATE']
        work_experience['company_name'] = company_name[0] if company_name else None
        work_experience['job_title'] = job_title[0] if job_title else None
        work_experience['employment_dates'] = employment_dates[0] if employment_dates else None
        work_experiences.append(work_experience)

    return work_experiences


experince = extract_work_experiences(text)
print(experince)
extracted_text["experince"] = experince

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_lincence_certifications(text):
    doc = nlp(text)
    certifications = []
    for ent in doc.ents:
        if ent.label_ == "LICENSE":
            title = ""
            organization = ""
            # Example logic to extract title and organization from license entity
            for token in ent:
                if token.ent_type_ == "TITLE":
                    title = token.text
                elif token.ent_type_ == "ORG":
                    organization = token.text
            certifications.append({
                "title": title,
                "issuingOrganization": organization
            })
    return certifications



certifications = extract_lincence_certifications(text)
print(certifications)
extracted_text["certifications"] = certifications

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def extract_license_certifications(text):
    certifications = []
    sentences = sent_tokenize(text)
    for sent in sentences:
        tree = ne_chunk(pos_tag(word_tokenize(sent)), binary=False)
        for subtree in tree.subtrees():
            if subtree.label() == "S" and "CERTIFICATION" in [x[0].upper() for x in subtree.leaves()]:
                # match patterns likely to correspond to the title and organization fields
                matches = re.findall(r'(?i)\b(?:CERTIFICATION|LICENSE)[S]?\b(?:\s+IN)?\s+([\w.,\s]+)\s*(?:-|–)?\s*([\w.,\s-]+)?', sent)
                if matches:
                    title, organization = matches[0]
                    certifications.append({
                        "title": title.strip(),
                        "organization": organization.strip() if organization else ''
                    })
    return certifications

certifications = extract_license_certifications(text)
print({"certifications": certifications})

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_license_cert(text):
    certifications = []
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "CERTIFICATE":
            title = ent.text
            organization = ""
            for tok in ent.subtree:
                if tok.ent_type_ == "ORG":
                    organization += tok.text
                else:
                    organization += " "
            certifications.append({
                "title": title.strip(),
                "organization": organization.strip()
            })
    return certifications

cert = extract_license_cert(text)
print({"certifications": cert})

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')


In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')
text = "LICENSE  & CERTIFICATIONS Python Certificate – HackerRank"
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append({'text': ent.text, 'label': ent.label_})
    return entities

entities = extract_entities(text)
print(entities)


In [ ]:
import spacy
import re

nlp = spacy.load('en_core_web_sm')

def extract_certifications(resume_text):
    doc = nlp(resume_text)
    matcher = Matcher(nlp.vocab)
    pattern = [{'LOWER': {'IN': ['certification', 'license']}}, {'IS_PUNCT': True, 'OP': '?'}, {'ENT_TYPE': {'NOT_IN': ['', 'PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']}}]
    matcher.add('CERT_ORG', None, pattern)
    matches = matcher(doc)
    certifications = []
    for match_id, start, end in matches:
        organization = doc[start:end].text
        # extract title of certification
        title = re.findall(r'(?i)\b(?:CERTIFICATION|LICENSE)[S]?\b(?:\s+IN)?\s+([\w.,\s]+)\s*(?:-|–)?\s*([\w.,\s-]+)?', doc[start:end].sent.text)
        if title:
            title = ' '.join(title[0])
        else:
            title = None
        certifications.append({'title': title, 'organization': organization})
    return certifications


In [ ]:
import re

def extract_portfolio_urls(resume_text):
    nlp_text = nlp(resume_text)
    portfolio_urls = []
    for sent in nlp_text.sents:
        for match in re.finditer("(?P<url>https?://[^\s]+)", sent.text):
            url = match.group("url")
            if "github" in url or "google drive" in url or "bitbucket" in url or "gitlab" in url:
                portfolio_urls.append(url)
    return portfolio_urls

resume_text = "My Github profile can be found at https://github.com/johndoe. Check out my projects on Google Drive at https://drive.google.com/drive/folders/123. You can also find my work on Bitbucket at https://bitbucket.org/johndoe."
portfolio_urls = extract_portfolio_urls(resume_text)
print(portfolio_urls)


In [ ]:
extracted_text